In [1]:
# !pip install torch

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
from sklearn.metrics import (r2_score, 
                             mean_squared_error, 
                             mean_absolute_error, 
                             mean_absolute_percentage_error)

from tqdm.auto import tqdm
import copy

np.random.seed(12345)
torch.manual_seed(123)

# ========== Data ==========

In [3]:
data = pd.read_csv('dataset.csv')

In [4]:
data

,x1,x2,y
0,2,1,9
1,2,-1,1
2,-2,2,6
3,1,2,11
4,-2,3,9
5,2,0,5
6,-1,-1,-2
7,-2,1,3
8,0,0,2
9,1,-1,0


In [5]:
X = data[['x1', 'x2']].to_numpy()

In [6]:
y = data['y'].to_numpy()

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, shuffle=True)

# ========== Model ==========

In [8]:
if torch.cuda.is_available():
  device = 'cuda:0'
else:
  device = 'cpu'

In [9]:
X_train = torch.from_numpy(X_train).float().to(device)
y_train = torch.from_numpy(y_train).float().to(device)

X_val = torch.from_numpy(X_val).float().to(device)
y_val = torch.from_numpy(y_val).float().to(device)

In [10]:
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [11]:
best_mse = np.inf

dropout_rates = [0, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

for dropout_rate in tqdm(dropout_rates):
    layers = []

    layers.append(nn.Linear(X.shape[1], 5))
    layers.append(nn.ReLU())
    layers.append(nn.Dropout(dropout_rate))

    layers.append(nn.Linear(5, 4))
    layers.append(nn.ReLU())
    layers.append(nn.Dropout(dropout_rate))

    layers.append(nn.Linear(4, 1))

    model = nn.Sequential(*layers).to(device)
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    loss_function = nn.MSELoss()
    for i in tqdm(range(500)):
        model.train()
        optimizer.zero_grad()

        y_pred_train = model(X_train)
        y_pred_train = y_pred_train.reshape(-1)

        loss = loss_function(y_pred_train, y_train)
        loss.backward()

        optimizer.step()
        
    with torch.no_grad():
        model.eval()
        y_pred_val = model(X_val)
        
    if np.isnan(y_pred_val.min()):
        continue
        
    mse = mean_squared_error(y_val, y_pred_val)
    
    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model)
        best_dropout_rate = dropout_rate

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

In [12]:
best_dropout_rate

0

In [13]:
for name, param in best_model.named_parameters():
    print(name, param)
    print()

0.weight Parameter containing:
tensor([[-0.3541, -0.1224],
        [ 0.1480,  1.0359],
        [ 0.2288,  0.9500],
        [-0.7965, -1.0297],
        [ 0.8646,  0.8920]], requires_grad=True)

0.bias Parameter containing:
tensor([-0.2008,  0.7424, -0.3487, -0.0506, -0.4535], requires_grad=True)

3.weight Parameter containing:
tensor([[-0.0889, -0.2331,  0.3379, -0.1050,  0.2827],
        [ 0.3204,  0.0442,  0.1027,  0.5257, -0.2036],
        [ 0.1714, -0.4011,  0.2555,  1.0016, -0.3893],
        [ 0.3955,  1.1498,  0.2942, -0.0481,  1.1700]], requires_grad=True)

3.bias Parameter containing:
tensor([-0.0214, -0.3222,  0.5109, -0.0267], requires_grad=True)

6.weight Parameter containing:
tensor([[ 0.1506, -0.2313, -1.1256,  1.5660]], requires_grad=True)

6.bias Parameter containing:
tensor([0.7895], requires_grad=True)



# ========== Prediction ==========

In [14]:
X = pd.read_csv('incoming_data.csv').to_numpy()

In [15]:
X = torch.from_numpy(X).float().to(device)

In [16]:
with torch.no_grad():
    best_model.eval()
    y_pred = best_model(X)
    
y_pred

tensor([[12.9875],
        [-2.9375],
        [ 5.0299],
        [-0.9340]])